### Notebook for preprocessing and cleaning data

Following steps are performed on the data:
1. Find and process any null columns or rows.
2. Find and process any duplicate columns or rows.


In [161]:
## Imports
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [162]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

print(f"Train {train_df.shape}")
print(f"Test {test_df.shape}")
train_df.head()

Train (10886, 12)
Test (6493, 9)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


### Convert datetime and added seprate features for month, year and hour

In [163]:
## Lets convert datetime column into datetime type
# Train
train = train_df.copy()
train.datetime = pd.to_datetime(train.datetime)
train['year'] = train.datetime.dt.year  # add year columns
train['month'] = train.datetime.dt.month # add month column
train['day'] = train.datetime.dt.day # add day of the month column
train['hour'] = train.datetime.dt.hour # add hour
train.drop('datetime', axis=1, inplace=True) # drop datetime column
print(f"Train {train.shape}")
# Test
# Store the test datetime to be used for submissions
test = test_df.copy()
test_datetime = test.datetime.copy()
test.datetime = pd.to_datetime(test.datetime)
test['year'] = test.datetime.dt.year  # add year columns
test['month'] = test.datetime.dt.month # add month column
test['day'] = test.datetime.dt.day # add day of the month column
test['hour'] = test.datetime.dt.hour # add hour
test.drop('datetime', axis=1, inplace=True) # drop datetime column
print(f"Test {test.shape}")
train.head()

Train (10886, 15)
Test (6493, 12)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
0,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2
3,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3
4,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4


### NaN values

In [164]:
print(f"Does train data have any row have NaN values: {train.isnull().any().any()}")
print(f"Does test data have any row have NaN values: {test.isnull().any().any()}")

Does train data have any row have NaN values: False
Does test data have any row have NaN values: False


### Duplicate row or columns

In [165]:
# duplicate rows
print(f"Train duplicate rows: {train.duplicated().any()}")
print(f"Test duplicate rows: {train.duplicated().any()}")
print()
# duplicate columns
print(f"Train duplicate columns: {train.T.duplicated().any()}")
print(f"Test duplicate columns: {train.T.duplicated().any()}")

Train duplicate rows: False
Test duplicate rows: False

Train duplicate columns: False
Test duplicate columns: False


In [166]:
train.dtypes

season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
year            int64
month           int64
day             int64
hour            int64
dtype: object

### Create processed dataframes

In [173]:
print(train.shape, test.shape)
train.to_csv('../input/train_processed.csv', index=False)
test.to_csv('../input/test_processed.csv', index=False)

(10886, 15) (6493, 12)


### Normalize data

In [174]:
# storing some fields for later
rental_counts = train[['casual', 'registered', 'count']].copy()
train.drop(['casual', 'registered', 'count'], axis=1, inplace=True)
train_columns = train.columns.tolist() # copy columns
test_columns = test.columns.tolist()

In [175]:
scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

In [176]:
print(train[:3])

[[0.         0.         0.         0.         0.2244898  0.30506768
  0.81       0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.20408163 0.28806354
  0.8        0.         0.         0.         0.         0.04347826]
 [0.         0.         0.         0.         0.20408163 0.28806354
  0.8        0.         0.         0.         0.         0.08695652]]


### Adding data back and creating dataframe

In [180]:
train_normalized = pd.DataFrame(train, columns = train_columns)
test_normalized = pd.DataFrame(test, columns = test_columns)
train_normalized['count'] = train_df['count']
train_normalized.to_csv('../input/train_normalised.csv', index=False)
test_normalized.to_csv('../input/test_normalised.csv', index=False)
train_normalized.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,hour,count
0,0.0,0.0,0.0,0.0,0.224490,0.305068,0.81,0.0,0.0,0.0,0.0,0.000000,16
1,0.0,0.0,0.0,0.0,0.204082,0.288064,0.80,0.0,0.0,0.0,0.0,0.043478,40
2,0.0,0.0,0.0,0.0,0.204082,0.288064,0.80,0.0,0.0,0.0,0.0,0.086957,32
3,0.0,0.0,0.0,0.0,0.224490,0.305068,0.75,0.0,0.0,0.0,0.0,0.130435,13
4,0.0,0.0,0.0,0.0,0.224490,0.305068,0.75,0.0,0.0,0.0,0.0,0.173913,1
